In [ ]:
import tweepy
import time
import random
import datetime
from datetime import timedelta
from access import * #import Twitter API from access.py
from googlesheet_keyword2 import * ##call googlesheet
import time

sleep_on_rate_limit=False 


# Call latest mentioned ID
id_file = 'last_seen_id.txt'

def retrieve_id(file):
    IDs_read = open(file, "r")
    last_id = IDs_read.read().strip() # print("Last ID was " + str(last_id))
    IDs_read.close()
    return last_id


def write_last_id(id, file):
    IDs_write = open(file, "w")
    IDs_write.write(str(id))
    IDs_write.close()
    return


# print keyword list
keyword_list = worksheet.range('B3:B52')
print('<키워드>')
KW=[]
for i in range(3, 27): #range(3, final n +5)
    locals()['KW_{}'.format(i)] = worksheet.acell('B%d' % (i)).value
    #print(eval('KW_'+str(i)))
    KW.append(worksheet.acell('B%d' % (i)).value)
    i=i+1
print(KW)


def auto_bot():
    print(mention.full_text)  # print the mention's text
    k=3
    for k in range(3, 24): #range(3, final n +2)
        #Tweet + Time(Korea)
        now = datetime.datetime.now()
        korea_time = now + timedelta(hours=9)
        t = str(korea_time.strftime("%Y-%m-%d %H:%M:%S"))
        if KW[k] in mention.full_text: # if mentioned tweet include keyword
            print("ID for the mention above: " + str(mention.id))
            row_data = worksheet.row_values(k+3)[2:] # keyword answer
            print(row_data)
            last_scanned_id = mention.id
            write_last_id(last_scanned_id, id_file)
            AN = random.choice(row_data)
            try:
                api.update_status("@" + mention.user.screen_name + " " + AN + "\n\n[" + t
                                              + "]", in_reply_to_status_id=mention.id)
            except tweepy.errors.Forbidden:
                pass
            print("Replied to @" + mention.user.screen_name + ' ' + AN)
            target_content = "True"
            break
            
        elif k<23 : # k < final n + 1
            print("@" + mention.user.screen_name + ' 의 답변에서 '
                  + 'B%d번째 ' % (k) + '키워드를 찾을 수 없음')
            target_content = "True"
            k = k+1

        else: # else, tweet disclude keyword
            print("@" + mention.user.screen_name + ' 의 답변에서 '
                  + '키워드가 존재하지 않음')
            last_scanned_id = mention.id
            write_last_id(last_scanned_id, id_file)
            target_content = "True"
            break
        
# Final
while True:
    # go through new mentions and scan them for a tag
    last_scanned_id = retrieve_id(id_file)
    mentions = api.mentions_timeline(tweet_mode="extended")
    if not last_scanned_id:  # Check if there is a stored # ID
        for mention in mentions:
            # Reply to tweets with specified tag/text
            target_content = "False"
            while target_content == "False":
                auto_bot()

    print("last scanned ID was " + str(last_scanned_id))
    # Only scan mentions after last scanned mention
    mentions = api.mentions_timeline(
        since_id=last_scanned_id, tweet_mode="extended")
    for mention in reversed(mentions):
        # Reply to tweets with specified tag/text
        auto_bot()
    time.sleep(30)  # To avoid rate limiting
    
!nohup jupyter notebook &